# Notebook App UI Example

This notebook demonstrates the in-notebook app interface letting you interact with a langchain app inside this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

from trulens.utils.keys import check_keys

check_keys(
    "OPENAI_API_KEY"
)

from trulens.core import Tru
from trulens.dashboard.appui import AppUI

tru = Tru()
tru.reset_database() # if needed
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

## langchain example

In [ ]:
from langchain_community.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI(temperature=0.9, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
)

# Conversational app puts it all together.
app = ConversationChain(
    llm=llm,
    memory=memory
)

from langchain.prompts import PromptTemplate
from trulens.instruments import instrument
instrument.method(PromptTemplate, "format")

truchain = tru.Chain(app, app_id="langchain_app")

In [ ]:
# Interesting Select.App queries to look at:
# - app.memory.chat_memory.messages[:].content
# - app.memory.moving_summary_buffer
# - app.prompt.template

# Interesting Select.Record queries to look at:
# - app.memory.save_context[0].args
# - app.prompt.format.args.kwargs
# - app.prompt.format.rets
# The last two need to instrument PromptTemplate as above.

aui = AppUI(
    app=truchain,
    
    app_selectors=[
        "app.memory.chat_memory.messages[:].content",
        "app.memory.moving_summary_buffer",

        "app.prompt.template"
    ],
    record_selectors=[
        "app.memory.save_context[0].args",

        "app.prompt.format.args.kwargs",
        "app.prompt.format.rets"
    ]
)
aui.widget

## llama_index example

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(
    html_to_text=True
).load_data(["http://paulgraham.com/worked.html"])
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

trullama = tru.Llama(query_engine, app_id="llama_index_app")

In [ ]:
aui = AppUI(
    app=trullama,
    
    app_selectors=[
    ],
    record_selectors=[
        "app.retriever.retrieve[0].rets[:].score",
        "app.retriever.retrieve[0].rets[:].node.text",
    ]
)
aui.widget

## basic app example

In [ ]:
def custom_application(prompt: str) -> str:
    return f"a useful response to {prompt}"

trubasic = tru.Basic(custom_application, app_id="basic_app")

In [ ]:
aui = AppUI(
    app=trubasic,
    
    app_selectors=[ # nothing interesting to display here
    ],
    record_selectors=[
        "app._call[0].args.args[:]",
    ]
)
aui.widget

## custom app example

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp # our custom app

# Create custom app:
app = CustomApp()

# Create trulens wrapper:
trucustom = tru.Custom(
    app=app,
    app_id="custom_app",
    
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app.respond_to_query
)


In [ ]:
aui = AppUI(
    app=trucustom,
    
    app_selectors=[
        "app.memory.messages[:]"
    ],
    record_selectors=[
        "app.retriever.retrieve_chunks[0].rets",
    ]
)
aui.widget